In [3]:
import numpy as np
import pandas as pd
import re
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

In [4]:
train_data = pd.read_csv('/content/drive/MyDrive/kaggle/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/kaggle/train.csv')
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
PassengerId = test_data['PassengerId']

In [20]:
full_data = [train_data, test_data]

train_data['Name_length'] = train_data['Name'].apply(len)
test_data['Name_length'] = test_data['Name'].apply(len)

y = lambda x: 0 if type(x) == float else 1

train_data['Has_Cabin'] = train_data['Cabin'].apply(y)
test_data['Has_Cabin'] = test_data['Cabin'].apply(y)

type(train_data['Cabin'][0])

float

In [22]:
for dataset in full_data:
  dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

for dataset in full_data:
  dataset['IsAlone'] = 0
  dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

for dataset in full_data:
  dataset['Embarked'] = dataset['Embarked'].fillna('S')

for dataset in full_data:
  dataset['Fare'] = dataset['Fare'].fillna(train_data['Fare'].median())

train_data['CategoricaalFare'] = pd.qcut(train_data['Fare'], 4)

for dataset in full_data:
  age_avg = dataset['Age'].mean()
  age_std = dataset['Age'].std()
  age_null_count = dataset['Age'].isnull().sum()
  age_null_rnd_lst = np.random.randint(
      age_avg - age_std, age_avg + age_std, size=age_null_count
  )
  dataset['Age'][np.isnan(dataset['Age'])] = age_null_rnd_lst
  dataset['Age'] = dataset['Age'].astype(int)

train_data['CategoricaalFare'] = pd.cut(train_data['Age'], 5)

In [23]:
train_data['CategoricaalFare']

0      (16.0, 32.0]
1      (32.0, 48.0]
2      (16.0, 32.0]
3      (32.0, 48.0]
4      (32.0, 48.0]
           ...     
886    (16.0, 32.0]
887    (16.0, 32.0]
888    (16.0, 32.0]
889    (16.0, 32.0]
890    (16.0, 32.0]
Name: CategoricaalFare, Length: 891, dtype: category
Categories (5, interval[float64, right]): [(-0.08, 16.0] < (16.0, 32.0] < (32.0, 48.0] <
                                           (48.0, 64.0] < (64.0, 80.0]]